# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Инструкция по выполнению проекта

1. Загрузите и подготовьте данные. Поясните порядок действий.  
2. Обучите и проверьте модель для каждого региона:  
2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.  
2.2. Обучите модель и сделайте предсказания на валидационной выборке.  
2.3. Сохраните предсказания и правильные ответы на валидационной выборке.  
2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.  
2.5. Проанализируйте результаты.  
  
3. Подготовьтесь к расчёту прибыли:  
3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.  
3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.  
3.3. Напишите выводы по этапу подготовки расчёта прибыли. 
  
4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:  
4.1. Выберите скважины с максимальными значениями предсказаний.  
4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.  
4.3. Рассчитайте прибыль для полученного объёма сырья.  
  
5. Посчитайте риски и прибыль для каждого региона:  
5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.  
5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.  
5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.  

# Описание данных

Данные геологоразведки трёх регионов находятся в файлах:  
/datasets/geo_data_0.csv  
/datasets/geo_data_1.csv  
/datasets/geo_data_2.csv  
id — уникальный идентификатор скважины;  
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
product — объём запасов в скважине (тыс. баррелей).  

# Условия задачи

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).  
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Ипортирование библиотек

In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
pd.options.display.max_columns = None
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

## Загрузка и подготовка данных

### Изучение данных

In [2]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


По всей видимости, никаких пропусков. В каждой строке текстовый id и 4 числовых значения, включая "product" - объем запасов в скважине

In [6]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [7]:
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
df0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [10]:
df1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
df2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Попробуем установить зависимости через корреляцию.

In [12]:
df0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


В регионе 0 наблюдается значимая линейная зависимость целевого признака product с каждым признаков f0, f1, f2. Наиболее сильная линейная зависимость наблюдается у признака f2.

In [13]:
df1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


В регионе 1 наблюдается почти абсолютная связь признака f2 с целевым признаком product.

In [14]:
df2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


В регионе 2 значимую линейную зависимость с целевым признаком можно наблюдать у признака f2.

Таким образом, по всем трем регионам наиболее значимым признаком для объема запасов в скважине является f2.

## Обучение и проверка модели

In [15]:
def learn_and_check(data):
    target = data['product']
    features = data.drop(['product','id'], axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, 
        target, 
        test_size=0.25, 
        random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid = pd.Series(predicted_valid, index=target_valid.index)
    mean_value = predicted_valid.mean()
    rmse_value = mean_squared_error(target_valid, predicted_valid) ** 0.5
    print('Среднее предсказанных значений = ', mean_value)
    print('RMSE предсказанных значений = ', rmse_value)
    
    return mean_value, rmse_value, target_valid, predicted_valid

In [16]:
mean_df0, rmse_df0, target_df0, predicted_valid_df0 = learn_and_check(df0)

Среднее предсказанных значений =  92.59256778438038
RMSE предсказанных значений =  37.5794217150813


In [17]:
mean_df1, rmse_df1, target_df1, predicted_valid_df1 = learn_and_check(df1)

Среднее предсказанных значений =  68.728546895446
RMSE предсказанных значений =  0.893099286775616


In [18]:
mean_df2, rmse_df2, target_df2, predicted_valid_df2 = learn_and_check(df2)

Среднее предсказанных значений =  94.96504596800489
RMSE предсказанных значений =  40.02970873393434


В нашем случае результаты получились таким образом, что чем выше среднее предсказанных значений, тем больше ошибка. Наиболее точные результаты предсказаний у региона №1, однако среднее значение наименьшее. Набольшее среднее значение у региона №2, однако и ошибка предсказаний наиболее большая.

## Подготовка к расчёту прибыли

Итак, нам известно, что product — объём запасов в скважине (тыс. баррелей). Доход с 1 ед. продукта = 450 тыс. рублей. Бюджет на всю разработку 10 млрд руб. Всего скважин - 200.

Рассчитаем минимальный суммарный объем, необходимый для покрытия расходов.

In [1]:
REVENUE_PER_PRODUCT = 450000
INVESTMENT_BUDGET = 10 ** 10
SELECTED_WELLS = 500
USED_WELLS = 200

In [20]:
min_volume_product = INVESTMENT_BUDGET / REVENUE_PER_PRODUCT
print(min_volume_product)

22222.222222222223


Суммарный объем должен составлять 22 222 тыс. баррелей. Соответственно рассчитаем минимальный средний объем одной из 200 скважин:

In [21]:
mean_min_product = min_volume_product / USED_WELLS
print(mean_min_product)

111.11111111111111


Сравним со средним каждого региона

In [35]:
def comparison(data):
    global mean_min_product
    mean_value = data['product'].mean()
    print('Средний объем скважины в регионе: ', mean_value)
    if mean_value < mean_min_product:
        print('Средний объем скважины недостаточен для покрытия расходов. Нам нужны лучшие.')
        print('Cредний объем скважины в регионе меньше нужного на:', mean_min_product - mean_value)

Регион №0

In [36]:
comparison(df0)

Средний объем скважины в регионе:  92.50000000000001
Средний объем скважины недостаточен для покрытия расходов. Нам нужны лучшие.
Cредний объем скважины в регионе меньше нужного на: 18.6111111111111


Регион №1

In [37]:
comparison(df1)

Средний объем скважины в регионе:  68.82500000000002
Средний объем скважины недостаточен для покрытия расходов. Нам нужны лучшие.
Cредний объем скважины в регионе меньше нужного на: 42.2861111111111


Регион №2

In [38]:
comparison(df2)

Средний объем скважины в регионе:  95.00000000000004
Средний объем скважины недостаточен для покрытия расходов. Нам нужны лучшие.
Cредний объем скважины в регионе меньше нужного на: 16.11111111111107


In [39]:
def profit_calculation(target, predictions, count):
    global REVENUE_PER_PRODUCT, INVESTMENT_BUDGET
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index].head(count)
    real_profit = selected.sum() * REVENUE_PER_PRODUCT - INVESTMENT_BUDGET
    
    return real_profit

## Расчёт прибыли и рисков 

Создадим функцию для проведения процедуры bootstrap

In [49]:
def region_profit(predictions, target):
    global SELECTED_WELLS, USED_WELLS
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000):
        target_subsample = target.sample(n=SELECTED_WELLS, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        profit = profit_calculation(target_subsample, probs_subsample, USED_WELLS)         
        values.append(profit)

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    risk = (values < 0).mean()

    mean = values.mean()
    print("Средняя прибыль: ", mean.round())
    print("95%-й доверительный интервал: ", [lower.round(), upper.round()])
    print("Риск получения убытков: ", risk)

In [51]:
region_profit(predicted_valid_df0, target_df0)

Средняя прибыль:  425938527.0
95%-й доверительный интервал:  [-102090095.0, 947976353.0]
Риск получения убытков:  0.06


In [52]:
region_profit(predicted_valid_df1, target_df1)

Средняя прибыль:  515222773.0
95%-й доверительный интервал:  [68873225.0, 931547591.0]
Риск получения убытков:  0.01


In [53]:
region_profit(predicted_valid_df2, target_df2)

Средняя прибыль:  435008363.0
95%-й доверительный интервал:  [-128880547.0, 969706954.0]
Риск получения убытков:  0.064


По региону №1 сформирована наиболее точная модель. В регионе №1 потенциально может быть наибольшая прибыль и наименьший риск. С учетом этого, там и следует начинать разработку.